In [1]:
using Distributed

In [2]:
@everywhere using Printf, SparseArrays, LinearAlgebra
@everywhere using IterativeSolvers, BenchmarkTools, Statistics
@everywhere using WriteVTK, Distributed
@everywhere using AbaqusReader, PyCall, StatsBase, JLD

In [3]:
@everywhere include("adiff.jl")
@everywhere include("materials.jl")
@everywhere include("elements.jl")

In [4]:
Elements.setp(16)

16

 using Printf, SparseArrays, LinearAlgebra
 using IterativeSolvers, BenchmarkTools, Statistics
 using WriteVTK, Distributed
 using AbaqusReader, PyCall, StatsBase, JLD

 include("adiff.jl")
 include("materials.jl")
 include("elements.jl")

In [5]:
@everywhere function cnst_func(u,ucntr,r0,cntr0,Rsq)
  r    = r0+u
  cntr = cntr0+ucntr
  sum((r-cntr).^2)-Rsq
end

In [6]:
function find_centre(nodes; dTol = 1e-12, maxiter=21)

  c0    = mean(nodes)  
  bdone = false  
  iter  = 0

  while !bdone
    D2c0   = adiff.D2(c0)
    global deltas = [(D2c0[1]-p[1]).^2+(D2c0[2]-p[2]).^2 for p in nodes]
    r      = norm(deltas)
    grad   = adiff.grad(r)
    if (norm(grad)<dTol) || (iter>maxiter)
      bdone = true
    else
      c0  -= adiff.hess(r)\grad
    end
    iter +=1
  end
  Rsquares = adiff.val.(deltas) 
  return (c0, Rsquares)
end

find_centre (generic function with 1 method)

In [7]:
sMeshFile = "Pattern2D03FinerMesh02j.inp"
Nsteps    = 200
ϵn        = -0.25
bfside    = true
sPosFix   = "NH"
mat       = Materials.NeoHooke(10)
bVTKall   = true
maxiter   = 31
dTol      = 1e-6
LF        = range(1/Nsteps/10,1,length=Nsteps)
sVTKpath  = "./vtk_files/"
bprogress = false
bforcebc  = true
bwarmup   = true
blheqs    = false
bsheqs    = true
;

In [8]:
mymodel   = AbaqusReader.abaqus_read_mesh(sMeshFile)
@printf("\n\t loaded %s \n", sMeshFile); flush(stdout)

(xmin, xmax) = extrema([item[2][1] for item ∈ mymodel["nodes"]])
(ymin, ymax) = extrema([item[2][2] for item ∈ mymodel["nodes"]])

nodes    = [mymodel["nodes"][ii]    for ii ∈ 1:length(mymodel["nodes"])]
el_nodes = [mymodel["elements"][ii] for ii ∈ 1:length(mymodel["elements"])]

elems = []
for item ∈ el_nodes
if length(item)==3
  push!(elems, Elements.Tria(item, nodes[item], mat=mat))
elseif length(item)==4
  push!(elems, Elements.Quad(item, nodes[item], mat=mat))
end
end    

points   = hcat(nodes...)
cells    = [if length(nodes)==3
            MeshCell(VTKCellTypes.VTK_TRIANGLE, nodes)
          else length(nodes)==4
            MeshCell(VTKCellTypes.VTK_QUAD, nodes)
          end  for nodes in el_nodes ]

@show nNodes, nElems  = length(nodes), length(el_nodes)
@show Δx, Δy          = (xmax-xmin), (ymax-ymin)

nid_bndl = mymodel["node_sets"]["ID_L"]
nid_bndr = mymodel["node_sets"]["ID_R"]
nid_bndt = mymodel["node_sets"]["ID_T"]
nid_bndb = mymodel["node_sets"]["ID_B"]

nid_sh01 = mymodel["node_sets"]["ID_SH01"]
nid_sh02 = mymodel["node_sets"]["ID_SH02"]
nid_sh03 = mymodel["node_sets"]["ID_SH03"]
nid_sh04 = mymodel["node_sets"]["ID_SH04"]
nid_sh05 = mymodel["node_sets"]["ID_SH05"]
nid_sh06 = mymodel["node_sets"]["ID_SH06"]
nid_sh07 = mymodel["node_sets"]["ID_SH07"]
nid_sh08 = mymodel["node_sets"]["ID_SH08"]
nid_sh09 = mymodel["node_sets"]["ID_SH09"]

nid_shx  = [nid_sh01, nid_sh02, nid_sh03,
          nid_sh04, nid_sh05, nid_sh06,
          nid_sh07, nid_sh08, nid_sh09]
nSHs     = length(nid_shx)

nid_lh01 = mymodel["node_sets"]["ID_LH01"]
nid_lh02 = mymodel["node_sets"]["ID_LH02"]
nid_lh03 = mymodel["node_sets"]["ID_LH03"]
nid_lh04 = mymodel["node_sets"]["ID_LH04"]
nid_lh05 = mymodel["node_sets"]["ID_LH05"]
nid_lh06 = mymodel["node_sets"]["ID_LH06"]
nid_lh07 = mymodel["node_sets"]["ID_LH07"]
nid_lh08 = mymodel["node_sets"]["ID_LH08"]
nid_lh09 = mymodel["node_sets"]["ID_LH09"]
nid_lh10 = mymodel["node_sets"]["ID_LH10"]
nid_lh11 = mymodel["node_sets"]["ID_LH11"]
nid_lh12 = mymodel["node_sets"]["ID_LH12"]
nid_lh13 = mymodel["node_sets"]["ID_LH13"]
nid_lh14 = mymodel["node_sets"]["ID_LH14"]
nid_lh15 = mymodel["node_sets"]["ID_LH15"]
nid_lh16 = mymodel["node_sets"]["ID_LH16"]

nid_lhx  = [nid_lh01, nid_lh02, nid_lh03,
          nid_lh04, nid_lh05, nid_lh06,
          nid_lh07, nid_lh08, nid_lh09, nid_lh10,
          nid_lh11, nid_lh12, nid_lh13,
          nid_lh14, nid_lh15, nid_lh16]
nLHs     = length(nid_lhx)
# displacement boundary conditions
cntrs_s = [find_centre(nodes[idx]) for idx in nid_shx]
cntrs_l = [find_centre(nodes[idx]) for idx in nid_lhx]
nodes = vcat(nodes,getindex.(cntrs_s,1),getindex.(cntrs_l,1))
;

┌ Info: 11678 nodes found
└ @ AbaqusReader /u1/vian294/.julia/packages/AbaqusReader/sDEH0/src/parse_mesh.jl:103
┌ Info: Parsing elements. Type: CPS4. Topology: Quad4
└ @ AbaqusReader /u1/vian294/.julia/packages/AbaqusReader/sDEH0/src/parse_mesh.jl:147
┌ Info: Creating nset ID_L
└ @ AbaqusReader /u1/vian294/.julia/packages/AbaqusReader/sDEH0/src/parse_mesh.jl:181
┌ Info: Creating elset ID_L
└ @ AbaqusReader /u1/vian294/.julia/packages/AbaqusReader/sDEH0/src/parse_mesh.jl:181
┌ Info: Creating nset ID_R
└ @ AbaqusReader /u1/vian294/.julia/packages/AbaqusReader/sDEH0/src/parse_mesh.jl:181
┌ Info: Creating elset ID_R
└ @ AbaqusReader /u1/vian294/.julia/packages/AbaqusReader/sDEH0/src/parse_mesh.jl:181
┌ Info: Creating nset ID_T
└ @ AbaqusReader /u1/vian294/.julia/packages/AbaqusReader/sDEH0/src/parse_mesh.jl:181
┌ Info: Creating elset ID_T
└ @ AbaqusReader /u1/vian294/.julia/packages/AbaqusReader/sDEH0/src/parse_mesh.jl:181
┌ Info: Creating nset ID_B
└ @ AbaqusReader /u1/vian294/.julia/pack


	 loaded Pattern2D03FinerMesh02j.inp 
(nNodes, nElems) = (length(nodes), length(el_nodes)) = (11678, 10832)
(Δx, Δy) = (xmax - xmin, ymax - ymin) = (20.0, 20.0)


In [9]:
idxs  = LinearIndices(((2, nNodes+nSHs+nLHs)))
eqns  = Array{Elements.ConstEq}(undef, 0)
if bsheqs
eqns  = vcat(eqns,
             [ [ begin
                  r0      = nodes[nid]
                  cntr0   = nodes[nNodes+ii]
                  Rsq     = cntrs_s[ii][2][jj]
                  id_dofs = vcat(idxs[:,nid][:],idxs[:,nNodes+ii][:])
                  Elements.ConstEq(x->cnst_func(x[1:2],x[3:4],r0,cntr0,Rsq), 
                                   id_dofs,adiff.D2) 
                end  for (jj,nid) in enumerate(nid_ii)] 
              for (ii,nid_ii) in enumerate(nid_shx)]...)
end
if blheqs
    eqns = vcat(eqns,
                [ [ begin
                     r0      = nodes[nid]
                     cntr0   = nodes[nNodes+nSHs+ii]
                     Rsq     = cntrs_l[ii][2][jj]
                     id_dofs = vcat(idxs[:,nid][:],idxs[:,nNodes+nSHs+ii][:])
                     Elements.ConstEq(x->cnst_func(x[1:2],x[3:4],r0,cntr0,Rsq), 
                                      id_dofs,adiff.D2) 
                   end  for (jj,nid) in enumerate(nid_ii)] 
                 for (ii,nid_ii) in enumerate(nid_lhx)]... )
end
;

In [10]:
u     = zeros(2, nNodes+nSHs+nLHs)
ifree = trues(size(u))
u[:,nid_bndb] .= 0
u[1,nid_bndt]     .= 0 
u[2,nid_bndt]     .= ϵn*Δy
ifree[:,nid_bndb] .= false
ifree[:,nid_bndt] .= false
if !bfside
    u[1,nid_bndl]     .= 0 
    u[1,nid_bndr]     .= 0 
    ifree[1,nid_bndl] .= false
    ifree[1,nid_bndr] .= false
end
icnst         = .!ifree

if bwarmup
    u0        = 1e-4Δx*randn(2, nNodes+nSHs+nLHs)
    u0[icnst] .= 1e-3LF[1]u[icnst]
    @time Elements.solvestep!(elems, u0, ifree,
                              bprogress=false, eqns=eqns, λ = zeros(length(eqns)),
                              becho=true, maxiter=maxiter, dTol=dTol)
    u[ifree] .= u0[ifree]
end
;


iter:  1, norm0: 2.37e+00, normr/dTol: 1.99e+07, eltime: 10.96 sec. 
iter:  2, norm0: 6.57e-02, normr/dTol: 1.72e+07, eltime: 7.36 sec. 
iter:  3, norm0: 7.54e-05, normr/dTol: 4.44e+07, eltime: 7.13 sec. 
iter:  4, norm0: 1.56e-05, normr/dTol: 5.21e+03, eltime: 6.61 sec. 
iter:  5, norm0: 1.56e-05, normr/dTol: 9.97e-03, eltime: 2.76 sec. 
 40.614362 seconds (43.53 M allocations: 29.788 GiB, 9.16% gc time)


In [11]:
sFileName = splitext(sMeshFile)[1]*sPosFix
@printf("\n\t starting %s \n", sFileName); flush(stdout)
allus = Elements.solve(elems, u,
                     ifree=ifree, eqns=eqns,
                     LF=LF, bprogress=bprogress,
                     becho=!bprogress, bechoi=!bprogress, ballus=true, 
                     maxiter = maxiter, dTol=dTol)

rf_tot = [sum(item[1][2,nid_bndt])   for item in allus]
Δu_tot = [mean(item[1][2,nid_bndt])  for item in allus]
##
JLD.save(sFileName*".jld", "nodes", nodes, "elems", elems, "allus", allus,
       "points", points, "cells", cells, "LF", LF, "mat", mat,
       "Nsteps", Nsteps, "nid_bndl", nid_bndl, "nid_bndr", nid_bndr, 
       "nid_bndt", nid_bndt, "nid_bndb", nid_bndb, 
       "u", u, "rf_tot", rf_tot, "Δu_tot", Δu_tot,
       "cntrs_s", cntrs_s, "cntrs_l", cntrs_l,
       "nid_shx", nid_shx, "nid_lhx", nid_lhx,
       "Δx", Δx, "Δy", Δy, "nNodes", nNodes, "nElems", nElems,
       "sMeshFile", sMeshFile, "sFileName", sFileName, "sVTKpath", sVTKpath)
@printf("results written to %s\n", sFileName); flush(stdout)
;


	 starting Pattern2D03FinerMesh02jNH 

iter:  1, norm0: 2.48e+00, normr/dTol: 1.00e+06, eltime: 6.04 sec. 
iter:  2, norm0: 7.11e-02, normr/dTol: 8.64e+05, eltime: 6.13 sec. 
iter:  3, norm0: 1.57e-02, normr/dTol: 2.70e+03, eltime: 6.79 sec. 
iter:  4, norm0: 1.56e-02, normr/dTol: 1.55e-03, eltime: 2.87 sec. 
step  1 done in  4 iter, after 22.31 sec.

iter:  1, norm0: 3.29e+01, normr/dTol: 9.99e+05, eltime: 5.76 sec. 
iter:  2, norm0: 6.92e+00, normr/dTol: 9.90e+05, eltime: 5.59 sec. 
iter:  3, norm0: 5.63e-01, normr/dTol: 8.25e+05, eltime: 6.16 sec. 
iter:  4, norm0: 1.74e-01, normr/dTol: 1.67e+04, eltime: 5.70 sec. 
iter:  5, norm0: 1.73e-01, normr/dTol: 9.18e-01, eltime: 2.78 sec. 
step  2 done in  5 iter, after 25.98 sec.

iter:  1, norm0: 3.31e+01, normr/dTol: 9.97e+05, eltime: 5.90 sec. 
iter:  2, norm0: 7.03e+00, normr/dTol: 9.77e+05, eltime: 6.65 sec. 
iter:  3, norm0: 6.85e-01, normr/dTol: 6.81e+05, eltime: 6.87 sec. 
iter:  4, norm0: 3.32e-01, normr/dTol: 8.81e+03, eltime: 6

Internal error: encountered unexpected error in runtime:
InterruptException()
jl_mutex_unlock at /buildworker/worker/package_linux64/build/src/locks.h:138 [inlined]
jl_typeinf_end at /buildworker/worker/package_linux64/build/src/gf.c:2610
typeinf_ext at ./compiler/typeinfer.jl:577
typeinf_ext at ./compiler/typeinfer.jl:613
jfptr_typeinf_ext_1.clone_1 at /u1/vian294/julia/julia-1.1.0/lib/julia/sys.so (unknown line)
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2219
jl_apply at /buildworker/worker/package_linux64/build/src/julia.h:1571 [inlined]
jl_type_infer at /buildworker/worker/package_linux64/build/src/gf.c:277
jl_compile_method_internal at /buildworker/worker/package_linux64/build/src/gf.c:1819 [inlined]
jl_fptr_trampoline at /buildworker/worker/package_linux64/build/src/gf.c:1863
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2219
process_backtrace at ./errorshow.jl:602 [inlined]
Type at ./task.jl:15 [inlined]
run_work_thunk at /bui

completed in 5 minutes, 17 seconds, 380 milliseconds, 627 microseconds, 336 nanoseconds


Error encountered while saving "/u1/vian294/METMAT/AutoDiff/rev_paper1/Pattern2D03FinerMesh02jNH.jld".
Fatal error:


UndefVarError: UndefVarError: save not defined

In [21]:
if bVTKall 
    paraview_collection(sVTKpath*sFileName) do pvd
      for (ii, item) in enumerate(allus)
        (u0,rf) = item[1:2]
        F   = Elements.getinfo(elems,u0,info=:F)
        J   = [det(F)                       for F  in F]
        C   = [transpose(F)*F               for F  in F]
        E   = [0.5*(C-I)                    for C  in C]
        L3  = [(F[1]F[4]-F[2]F[3])^-2       for F  in F]
        Ic  = [Materials.getInvariants(C[ii], L3[ii]) for ii in 1:nElems] 

        vtkobj  = vtk_grid(@sprintf("%s_%03i", sVTKpath*sFileName, ii),
                           points+u0[:,1:nNodes], cells)
        vtk_point_data(vtkobj, (u0[1,:],u0[2,:]), "u")
        vtk_point_data(vtkobj, (rf[1,:],rf[2,:]), "rf")
        vtk_cell_data(vtkobj,  J,   "J")
        vtk_cell_data(vtkobj,  tuple([getindex.(E,ii)   for ii in 1:4]...), "E")
        vtk_cell_data(vtkobj,  tuple([getindex.(F,ii)   for ii in 1:4]...), "F")
        vtk_cell_data(vtkobj,  tuple([getindex.(Ic,ii)  for ii in 1:3]...), "I")

        collection_add_timestep(pvd, vtkobj, ii)
        vtk_save(vtkobj)
      end
    end  
else  
    (u0,rf) = allus[end][1:2]
    F   = Elements.getinfo(elems,u0,info=:F)
    J   = [det(F)                       for F  in F]
    C   = [transpose(F)*F               for F  in F]
    E   = [0.5*(C-I)                    for C  in C]
    L3  = [(F[1]F[4]-F[2]F[3])^-2       for F  in F]
    Ic  = [Materials.getInvariants(C[ii], L3[ii]) for ii in 1:nElems] 
    vtk_grid(@sprintf("%s_%03i", sVTKpath*sFileName, ii),
             points+u0[:,1:nNodes], cells) do vtkobj

    vtk_point_data(vtkobj, (u0[1,:],u0[2,:]), "u")
    vtk_point_data(vtkobj, (rf[1,:],rf[2,:]), "rf")
    vtk_cell_data(vtkobj,  J,   "J")
    vtk_cell_data(vtkobj,  tuple([getindex.(E,ii)   for ii in 1:4]...), "E")
    vtk_cell_data(vtkobj,  tuple([getindex.(F,ii)   for ii in 1:4]...), "F")
    vtk_cell_data(vtkobj,  tuple([getindex.(Ic,ii)  for ii in 1:3]...), "I")

    vtk_save(vtkobj)
    end
end

UndefVarError: UndefVarError: sFileName not defined